In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install altair 
import altair as alt
from scipy.stats import ttest_ind
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/districtwise-crimes-committed-against-women-2015/District-wise_Crimes_committed_against_Women_2015_1.csv
/kaggle/input/crime-against-women-20012014-india/crimes_against_women_2001-2014.csv


## Reading above file path into a dataframe

In [2]:
df = pd.read_csv('/kaggle/input/crime-against-women-20012014-india/crimes_against_women_2001-2014.csv')
df1 = pd.read_csv('/kaggle/input/districtwise-crimes-committed-against-women-2015/District-wise_Crimes_committed_against_Women_2015_1.csv')

In [3]:
df.info()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10677 entries, 0 to 10676
Data columns (total 11 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   Unnamed: 0                                           10677 non-null  int64 
 1   STATE/UT                                             10677 non-null  object
 2   DISTRICT                                             10677 non-null  object
 3   Year                                                 10677 non-null  int64 
 4   Rape                                                 10677 non-null  int64 
 5   Kidnapping and Abduction                             10677 non-null  int64 
 6   Dowry Deaths                                         10677 non-null  int64 
 7   Assault on women with intent to outrage her modesty  10677 non-null  int64 
 8   Insult to modesty of Women                           10677 non-null  int64 


In [4]:
print('df:', df.columns, '\n df1:',df1.columns)
df1.rename(columns={'Kidnapping & Abduction_Total':'Kidnapping and Abduction',
'Assault on Women with intent to outrage her Modesty_Total':'Assault on women with intent to outrage her modesty',
'Insult to the Modesty of Women_Total':'Insult to modesty of Women',
'Importation of Girls from Foreign Country':'Importation of Girls','State/ UT':'STATE/UT', 'District/ Area':'DISTRICT'}, inplace=True)

if 'Unnamed: 0' in df:
    df = df.drop('Unnamed: 0', axis=1)

print('df:', df.columns, '\ndf1:',df1.columns)


df: Index(['Unnamed: 0', 'STATE/UT', 'DISTRICT', 'Year', 'Rape',
       'Kidnapping and Abduction', 'Dowry Deaths',
       'Assault on women with intent to outrage her modesty',
       'Insult to modesty of Women', 'Cruelty by Husband or his Relatives',
       'Importation of Girls'],
      dtype='object') 
 df1: Index(['State/ UT', 'Sl. No.', 'District/ Area', 'Year', 'Rape',
       'Attempt to commit Rape', 'Kidnapping & Abduction_Total',
       'Dowry Deaths',
       'Assault on Women with intent to outrage her Modesty_Total',
       'Insult to the Modesty of Women_Total',
       'Cruelty by Husband or his Relatives',
       'Importation of Girls from Foreign Country',
       'Abetment of Suicides of Women', 'Dowry Prohibition Act, 1961',
       'Indecent Representation of Women (P) Act, 1986',
       'Protection of Children from Sexual Offences Act',
       'Protection of Women from Domestic Violence Act, 2005',
       'Immoral Traffic Prevention Act', 'Total Crimes against Women']

In [5]:
#removing the extra columns from 2015 dataset

df1 = df1[[col for col in df.columns]]
print(df1.columns)

Index(['STATE/UT', 'DISTRICT', 'Year', 'Rape', 'Kidnapping and Abduction',
       'Dowry Deaths', 'Assault on women with intent to outrage her modesty',
       'Insult to modesty of Women', 'Cruelty by Husband or his Relatives',
       'Importation of Girls'],
      dtype='object')


## Initial DataFrame Exploration

In [6]:
#last line only prints so have to comment out others and run cell to see each output

df.info()
df.head()

df.tail()
df.info()
df.describe()

print("Null values in column:")
for column in df.columns:
    print(column + ": " + str(df[column].isnull().sum()))

df1.info()
print("Null values in column:")
for column in df1.columns:
    print(column + ": " + str(df1[column].isnull().sum()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10677 entries, 0 to 10676
Data columns (total 10 columns):
 #   Column                                               Non-Null Count  Dtype 
---  ------                                               --------------  ----- 
 0   STATE/UT                                             10677 non-null  object
 1   DISTRICT                                             10677 non-null  object
 2   Year                                                 10677 non-null  int64 
 3   Rape                                                 10677 non-null  int64 
 4   Kidnapping and Abduction                             10677 non-null  int64 
 5   Dowry Deaths                                         10677 non-null  int64 
 6   Assault on women with intent to outrage her modesty  10677 non-null  int64 
 7   Insult to modesty of Women                           10677 non-null  int64 
 8   Cruelty by Husband or his Relatives                  10677 non-null  int64 


In [7]:
df = df.append(df1)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11530 entries, 0 to 852
Data columns (total 10 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   STATE/UT                                             11530 non-null  object 
 1   DISTRICT                                             11529 non-null  object 
 2   Year                                                 11529 non-null  float64
 3   Rape                                                 11529 non-null  float64
 4   Kidnapping and Abduction                             11529 non-null  float64
 5   Dowry Deaths                                         11529 non-null  float64
 6   Assault on women with intent to outrage her modesty  11529 non-null  float64
 7   Insult to modesty of Women                           11529 non-null  float64
 8   Cruelty by Husband or his Relatives                  11529 non-null 

In [8]:
# 2001-2012 had capital state names and 2013,2014 had lowercase
df['STATE/UT'] = df['STATE/UT'].apply(lambda x: x.lower())
for year in df['Year'].unique():
    if not (df[(df['STATE/UT']=='a&n islands') | (df['STATE/UT']=='a & n islands') & df['Year']==year].empty):
        print('AHA')
#combining 2 names for A&N islands, D&D haveli, delhi into 1
df['STATE/UT'] = df['STATE/UT'].apply(lambda x: 'a&n islands' if x in ['a & n islands','a&n islands'] else x)
df['STATE/UT'] = df['STATE/UT'].apply(lambda x: 'd&n haveli' if x in ['d & n haveli','d&n haveli','daman & diu'] else x)
df['STATE/UT'] = df['STATE/UT'].apply(lambda x: 'delhi' if x in ['delhi','delhi ut'] else x)

#2013,2014 had district total row named 'ZZ TOTAL', 'Total District(s)'. Changed to common name 'TOTAL'
#df[(df['STATE/UT']=='andhra pradesh') & (df['Year']==2007)]['DISTRICT']
df['DISTRICT'] = df['DISTRICT'].apply(lambda x: 'total' if x in ['ZZ TOTAL','Total District(s)','TOTAL'] else x)
#check 
#df[(df['STATE/UT']=='andhra pradesh') & (df['Year']==2014)]['DISTRICT'] 
print(df['STATE/UT'].unique())


['andhra pradesh' 'arunachal pradesh' 'assam' 'bihar' 'chhattisgarh' 'goa'
 'gujarat' 'haryana' 'himachal pradesh' 'jammu & kashmir' 'jharkhand'
 'karnataka' 'kerala' 'madhya pradesh' 'maharashtra' 'manipur' 'meghalaya'
 'mizoram' 'nagaland' 'odisha' 'punjab' 'rajasthan' 'sikkim' 'tamil nadu'
 'tripura' 'uttar pradesh' 'uttarakhand' 'west bengal' 'a&n islands'
 'chandigarh' 'd&n haveli' 'delhi' 'lakshadweep' 'puducherry' 'telangana']


In [9]:
df.head()


,STATE/UT,DISTRICT,Year,Rape,Kidnapping and Abduction,Dowry Deaths,Assault on women with intent to outrage her modesty,Insult to modesty of Women,Cruelty by Husband or his Relatives,Importation of Girls
0,andhra pradesh,ADILABAD,2001.0,50.0,30.0,16.0,149.0,34.0,175.0,0.0
1,andhra pradesh,ANANTAPUR,2001.0,23.0,30.0,7.0,118.0,24.0,154.0,0.0
2,andhra pradesh,CHITTOOR,2001.0,27.0,34.0,14.0,112.0,83.0,186.0,0.0
3,andhra pradesh,CUDDAPAH,2001.0,20.0,20.0,17.0,126.0,38.0,57.0,0.0
4,andhra pradesh,EAST GODAVARI,2001.0,23.0,26.0,12.0,109.0,58.0,247.0,0.0


In [10]:
#df[df['DISTRICT']=='Total District(s)']
#This will select 2015 ke records se total values alone. Ideally should rename this to total so it's consistent. Use the lambda x: stuff
df['DISTRICT'] = df['DISTRICT'].apply(lambda x: 'total' if x in ['Total District(s)'] else x)

# get the 'total' rows
df_state_totals_per_year = df[df['DISTRICT']=='total'] 
#print(df[df['Year']==2001][['STATE/UT', 'Rape']])
print(df_state_totals_per_year[df_state_totals_per_year['Year']==2015][ 'Rape'].sum())
# When you do df[['random', 'stuff']] you are selecting 2 columns callwed random and stuff.
# want to hold data per year So lets select 'total' rows first.


34651.0


In [11]:
#df.columns
#'STATE/UT', 'DISTRICT', 'Year', 'Rape', 'Kidnapping and Abduction','Dowry Deaths', 'Assault on women with intent to outrage her modesty','Insult to modesty of Women', 'Cruelty by Husband or his Relatives','Importation of Girls'
df_norm_crimes = pd.melt(df_state_totals_per_year, id_vars=['STATE/UT', 'Year'], value_vars=['Rape', 'Kidnapping and Abduction','Dowry Deaths', 'Assault on women with intent to outrage her modesty','Insult to modesty of Women', 'Cruelty by Husband or his Relatives','Importation of Girls'])

In [12]:
df_norm_crimes.head()

,STATE/UT,Year,variable,value
0,andhra pradesh,2001.0,Rape,871.0
1,arunachal pradesh,2001.0,Rape,33.0
2,assam,2001.0,Rape,817.0
3,bihar,2001.0,Rape,888.0
4,chhattisgarh,2001.0,Rape,959.0


In [13]:
df_norm_crimes[(df_norm_crimes['variable']=='Rape') & (df_norm_crimes['Year']==2015.0)]['value'].sum()

34651.0

In [14]:
chart = alt.Chart(df_norm_crimes).mark_bar().encode(
    x=alt.X('Year:O', title='Year'),
    y=alt.Y('sum(value):Q', title="Total number of crimes"),
    color=alt.Color('variable:N', title='Crimes'),
    tooltip='sum(value):Q'
).properties(height=600, width=600)
chart.save("Crimes.html")
#chart

# Conclusion
Sum of all crimes in each year while showing what part of the total (coloured differently) is because of that particular crime.
Overall the total number of crimes have risen slowly and then very fast over the years.

In [15]:
print(df_norm_crimes[df_norm_crimes['Year']==2001])

               STATE/UT    Year              variable  value
0        andhra pradesh  2001.0                  Rape  871.0
1     arunachal pradesh  2001.0                  Rape   33.0
2                 assam  2001.0                  Rape  817.0
3                 bihar  2001.0                  Rape  888.0
4          chhattisgarh  2001.0                  Rape  959.0
...                 ...     ...                   ...    ...
3119         chandigarh  2001.0  Importation of Girls    0.0
3120         d&n haveli  2001.0  Importation of Girls    0.0
3121         d&n haveli  2001.0  Importation of Girls    0.0
3122        lakshadweep  2001.0  Importation of Girls    0.0
3123         puducherry  2001.0  Importation of Girls    0.0

[238 rows x 4 columns]


In [16]:
#shortening crime names for better visibility in graphs
df_norm_crimes['variable'].unique()
new_var = {'Rape':'Rape',
           'Kidnapping and Abduction':'Kidnapping',
           'Dowry Deaths':'Dowry Deaths',
           'Assault on women with intent to outrage her modesty':'Assault',
           'Insult to modesty of Women':'Insult to modesty',
           'Cruelty by Husband or his Relatives':'Cruelty by Family',
           'Importation of Girls':'Importation'}
df_norm_crimes['variable'] = df_norm_crimes['variable'].apply(lambda x: new_var[x] if x in new_var else x)
df_norm_crimes['variable'].unique()

array(['Rape', 'Kidnapping', 'Dowry Deaths', 'Assault',
       'Insult to modesty', 'Cruelty by Family', 'Importation'],
      dtype=object)

In [17]:

chart = alt.Chart(df_norm_crimes).mark_boxplot().encode(
    x=alt.X('variable:N', title="Crimes"),
    y=alt.Y('sum(value):Q', title="Number of crimes"),
    color=alt.Color('variable:N', title='Crimes'),
    facet=alt.Facet('Year:O', columns=7)
).properties(width=300, height=300)
chart.save("Yearwise_crimes.html")

In [18]:
#splitting according to Nirbhaya Act - 2010-12 and 2013-15
df_bc13 = df_norm_crimes[(df_norm_crimes['Year'].isin([2010,2011,2012])) & (df_norm_crimes['variable']=='Rape')]
df_ad13 = df_norm_crimes[(df_norm_crimes['Year'].isin([2013,2014,2015])) & (df_norm_crimes['variable']=='Rape')]
ad_13 = []
bc_13 = []
    
for year in df_bc13['Year'].unique():
    print(year)
    bc_13.append(df_bc13[df_bc13['Year']==year]['value'].sum())
for year in df_ad13['Year'].unique():
    ad_13.append(df_ad13[df_ad13['Year']==year]['value'].sum())
    
print(bc_13)
print(ad_13)

2010.0
2011.0
2012.0
[21665.0, 23634.0, 24217.0]
[33707.0, 36735.0, 34651.0]


In [19]:
result = ttest_ind(bc_13, ad_13)

In [20]:
result
#If we observe a large p-value, for example larger than 0.05 or 0.1, 
#then we cannot reject the null hypothesis of identical average scores. 
#If the p-value is smaller than the threshold, e.g. 1%, 5% or 10%, then we reject the null hypothesis of equal averages.

#2001,2002,2003,2004,2005,2006,2007,2008,2009, included in bc_13 gives
#Ttest_indResult(statistic=-8.539030164242636, pvalue=1.0874193136267394e-06)
#p-value is even lower

Ttest_indResult(statistic=-10.036128581467946, pvalue=0.0005542104071428713)

In [21]:
len(df_ad13)


107

# Conclusion
Number of crimes of all categories are increasing from 2001 to 2014
Highest number of crimes against women is because of cruelty by the husband or relatives.
Close to this is Assault with an intention to outrage modesty of the woman.
Kidnapping and rape follow.

Cruelty by Family has the highest number of outliers throughout the years.
Next, a check on the states that are causing the outliers in Cruelty by Family category.

In [22]:
df_cruelty = df_norm_crimes[df_norm_crimes['variable']=='Cruelty by Family']
df_cruelty

,STATE/UT,Year,variable,value
2575,andhra pradesh,2001.0,Cruelty by Family,5791.0
2576,arunachal pradesh,2001.0,Cruelty by Family,11.0
2577,assam,2001.0,Cruelty by Family,1248.0
2578,bihar,2001.0,Cruelty by Family,1558.0
2579,chhattisgarh,2001.0,Cruelty by Family,840.0
...,...,...,...,...
3085,d&n haveli,2015.0,Cruelty by Family,2.0
3086,d&n haveli,2015.0,Cruelty by Family,3.0
3087,delhi,2015.0,Cruelty by Family,3521.0
3088,lakshadweep,2015.0,Cruelty by Family,2.0


In [23]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['STATE/UT'], empty='none')

# The basic line
line = alt.Chart(df_cruelty).mark_line(interpolate='monotone').encode(
    x='STATE/UT:N',
    y='value:Q',
    color='Year:O'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(df_cruelty).mark_point().encode(
    x='STATE/UT:N',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=10, dy=-10).encode(
    text=alt.condition(nearest, 'value:Q', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(df_cruelty).mark_rule(color='gray').encode(
    x='STATE/UT:N',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
chart = alt.layer(
    line, selectors, points, rules, text
).properties(
    width=1000, height=1000
)
chart.save("Statewise_crimes.html")
    

# Conclusion
West Bengal, Rajasthan and Andhra Pradesh have the highest values for most of the years in the range given

In [24]:
#to check if the values shown on the chart are correct
df_cruelty[df_cruelty['STATE/UT']=='west bengal'].max()

STATE/UT          west bengal
Year                   2015.0
variable    Cruelty by Family
value                 23278.0
dtype: object